In [1]:
from utils.Dataloader import load_movies,load_ratings,load_users
movies_df = load_movies('datasets/')
ratings_df = load_ratings('datasets/')
users_df = load_users('datasets/')
display(movies_df)
display(ratings_df)

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
import pandas as pd
# 영화 정보를 시청 내역과 매핑하여 출력
sorted_ratings_df = ratings_df.sort_values(by=['userId', 'timestamp'])

for user_id, user_data in sorted_ratings_df.groupby('userId'):
    print(f"User {user_id}'s Movie Watching History:")
    for index, row in user_data.iterrows():
        movie_id = row['movieId']
        movie_info = movies_df[movies_df['movieId'] == movie_id]
        if not movie_info.empty:
            title = movie_info['title'].values[0]
            genres = movie_info['genres'].values[0]
            timestamp = row['timestamp']
            print(f"Movie Title: {title}, Genres: {genres}, Timestamp: {timestamp}")
    print()


User 1's Movie Watching History:
Movie Title: Girl, Interrupted (1999), Genres: Drama, Timestamp: 978300019
Movie Title: Back to the Future (1985), Genres: Comedy|Sci-Fi, Timestamp: 978300055
Movie Title: Titanic (1997), Genres: Drama|Romance, Timestamp: 978300055
Movie Title: Cinderella (1950), Genres: Animation|Children's|Musical, Timestamp: 978300055
Movie Title: Meet Joe Black (1998), Genres: Romance, Timestamp: 978300103
Movie Title: Last Days of Disco, The (1998), Genres: Drama, Timestamp: 978300172
Movie Title: Erin Brockovich (2000), Genres: Drama, Timestamp: 978300275
Movie Title: Christmas Story, A (1983), Genres: Comedy|Drama, Timestamp: 978300719
Movie Title: To Kill a Mockingbird (1962), Genres: Drama, Timestamp: 978300719
Movie Title: One Flew Over the Cuckoo's Nest (1975), Genres: Drama, Timestamp: 978300760
Movie Title: Wallace & Gromit: The Best of Aardman Animation (1996), Genres: Animation, Timestamp: 978300760
Movie Title: Star Wars: Episode IV - A New Hope (1977), 

KeyboardInterrupt: 

In [6]:
import os
import numpy as np
import tensorflow as tf
from models import relation_matrix_ratings_50_256  # 모델을 정의한 파일의 경로를 import합니다.

# 모델을 저장한 디렉토리 경로
model_dir = "models"
model_checkpoint = os.path.join(model_dir, "model.ckpt")

# 모델 불러오기
tf.reset_default_graph()  # 그래프 초기화
sess = tf.Session()
model = Model(...)  # Model 클래스 생성자에 필요한 매개변수를 넣어줍니다.
saver = tf.train.Saver()
saver.restore(sess, model_checkpoint)

def recommend_movies_for_user(user_id, num_recommendations=5):
    # 사용자의 시퀀스를 설정합니다. 예시로 처음 몇 개의 아이템을 사용합니다.
    user_seq = [1, 3, 5, 7, 9]  # 예시 시퀀스
    user_seq_length = len(user_seq)

    # 사용자 시퀀스를 모델의 입력 형식에 맞게 변환합니다.
    input_user = np.array([user_id] * user_seq_length)
    input_seq = np.array([user_seq])
    input_time_seq = np.array([list(range(user_seq_length))])

    # 모델의 예측을 수행합니다.
    feed_dict = {
        model.u: input_user,
        model.input_seq: input_seq,
        model.time_matrix: input_time_seq,
        model.is_training: False
    }
    predicted_scores = sess.run(model.logits, feed_dict=feed_dict)

    # 예측 점수를 기반으로 영화 추천을 수행합니다.
    recommended_movie_indices = np.argsort(predicted_scores[0])[::-1][:num_recommendations]

    # 추천 영화의 인덱스를 실제 영화 ID로 변환합니다.
    recommended_movies = [movie_id_mapping[i] for i in recommended_movie_indices]

    return recommended_movies

# 특정 사용자에게 영화를 추천합니다.
user_id_to_recommend = 123  # 추천하고자 하는 사용자 ID
num_recommendations = 5  # 추천할 영화 개수
recommended_movies = recommend_movies_for_user(user_id_to_recommend, num_recommendations)
print("Recommended movies for user {}: {}".format(user_id_to_recommend, recommended_movies))

2.12.0
